In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import wandb
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [3]:
trainfile = "triples.train.10m.utf8.simpletransformers.tsv"
devfile   = "mydevset_triples.tsv"

train_df = pd.read_csv(trainfile, sep="\t", names=["text_a", "text_b", "labels"], nrows=2000000)
eval_df = pd.read_csv(devfile, sep="\t", names=["text_a", "text_b", "labels"])

In [4]:
model_args = ClassificationArgs()
model_args.lazy_loading = True
model_args.lazy_loading_start_line = 0
model_args.lazy_text_a_column = 0
model_args.lazy_text_b_column = 1
model_args.lazy_labels_column = 2
model_args.eval_batch_size = 16
model_args.dataloader_num_workers = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = 3000
model_args.save_eval_checkpoints = True
model_args.save_model_every_epoch = False
model_args.learning_rate = .0000274
model_args.manual_seed = 4
model_args.max_seq_length = 512
model_args.multiprocessing_chunksize = 5000
model_args.no_cache = False
model_args.no_save = False
model_args.num_train_epochs = 1
model_args.output_dir = "outputs_scibert"
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 8
model_args.gradient_accumulation_steps = 2
model_args.train_custom_parameter_only = False
model_args.n_gpu = 1
model_args.dataloader_num_workers = 1
model_args.labels_list = [0, 1]

In [5]:
def evalAcc(x,y):
    acc = sklearn.metrics.accuracy_score(x,[round(p) for p in y])
    wandb.log({"accuracy": acc})
    return acc

In [6]:
wandb.init()

model = ClassificationModel(
    "bert",
    "allenai/scibert_scivocab_cased",
    args=model_args
)
model.train_model(trainfile, eval_df=devfile, accuracy=evalAcc)

metrics = model.eval_model(devfile,
    accuracy=lambda x,y: sklearn.metrics.accuracy_score(x,[round(p) for p in y]))[0]
print(f"ACC: {metrics['accuracy']}")
wandb.log(metrics)

INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjIwM25uZDk4OnVuY2F0ZWdvcml6ZWQ6a2hpdHVyYXM=
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/config.yaml
INFO:filelock:Lock 139721321835792 acquired on /home/faessler/.cache/torch/transformers/560df3639836cbc0b55a7264963b1b5a7abc7ab307932944f88d56a79daf9f77.aaa68d18427cc99d0721bc3bfdda0f2cb193951b42ed051072952bf6639e3667.lock
INFO:wandb.run_manager:file/dir created: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-history.js

INFO:filelock:Lock 139721321835792 released on /home/faessler/.cache/torch/transformers/560df3639836cbc0b55a7264963b1b5a7abc7ab307932944f88d56a79daf9f77.aaa68d18427cc99d0721bc3bfdda0f2cb193951b42ed051072952bf6639e3667.lock


INFO:filelock:Lock 139719521453328 acquired on /home/faessler/.cache/torch/transformers/a34f6a7a07ff8f7723c97a1d264a218c75fbd9a27a3ee958bb194bec30478319.dab672d3d9c86398f8504e0e3f46391eca25af4f7b7a88bae481e9d6974e731f.lock


INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json
INFO:filelock:Lock 139719521453328 released on /home/faessler/.cache/torch/transformers/a34f6a7a07ff8f7723c97a1d264a218c75fbd9a27a3ee958bb194bec30478319.dab672d3d9c86398f8504e0e3f46391eca25af4f7b7a88bae481e9d6974e731f.lock


INFO:filelock:Lock 139719521438224 acquired on /home/faessler/.cache/torch/transformers/ae3febdd51990b429457018b5852c92e1be6bb95248faf4ac377d66fc4a5d8d0.a8e6b7905f755590e7ebfff6b58d35a8589e27e0ad0033165cb1bfd32dc9bbed.lock


...

INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /h

KeyboardInterrupt: 

INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:stopping streaming files and file change observer
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200802_171648-203nnd98/wandb-metadata.json


Error in callback <function _init_jupyter.<locals>.cleanup at 0x7f136f4407a0> (for post_run_cell):


KeyboardInterrupt: 